# Steganography

The goal is to hide a secret message in a jpeg. Steps we need to take:

1. Load an image as an array of bytes
2. Convert message into bits
3. For each bit in message, change the lsb of every pixel until the entire message is encoded

*Optional:* 
- Ensure the image is large enough to contain the message. 
- Choose how many bits to change per pixel
  - Distribute bit change equally on all channels

That's it for **encoding** the message, but we also need to **decode** it. To do that, we need to know the size of the message, so we need to make sure the message contains some header. 

With that out of the way, this is the image we will be working on:

<img src="nomorefanmail.jpg" width="500">

First we need to **encode the message in binary**:

In [183]:
def str_to_bin(str : str) -> str:
    # Different approaches to achieve the same thing: 
    # bits = [format(ord(char), '08b') for char in msg]                 # eg. ['01010111', '01100001', '01110010', '00100000']
    # from https://www.geeksforgeeks.org/python/python-convert-string-to-binary/ :
    # bits = ''.join(format(ord(char), '08b') for char in msg)          # eg. "01010111011000010111001000100000011000010110111001"
    bits = [bit for char in str for bit in format(ord(char), '08b')]    # eg. ['0', '1', '0', '1', '0', '1', '1', '1', '0']
    return bits

def bin_to_str(bits : list) -> str:
    # for every 8 elements in bits, join 8 bits to a "bitstring" and convert them to a unicode character
    chars = [chr(int(''.join(bits[i:i+8]), 2)) for i in range(0, len(bits), 8)]
    original_msg = ''.join(chars)
    return original_msg

Then, we'll **load the image**:

In [184]:
from PIL import Image
import numpy as np

def load_image(path : str):
    img = Image.open(path) 
    pixels = np.array(img, dtype=np.uint8) # this gives us a 2D array, pixels[0][0] gives us the data of the first pixel
    # example pixel: [224 238 241]
    return pixels

path = "nomorefanmail.jpg"
pixels = load_image(path)

We'll define a function that takes a pixel, and **hides one bit in one of the channels**.

In [185]:
def encode_on_pixel(pixel, msg_bit: str):
    # side effects: (none)
    
    if len(msg_bit) != 1:
        raise ValueError(f"Error: Invalid bit length: {len(msg_bit)}")
    
    if msg_bit != '1' and msg_bit != '0':
        raise ValueError(f"Error: Invalid message bit: {msg_bit}")
    
    red_channel, blue_channel, green_channel = pixel

    binary = list(f'{blue_channel:08b}') # Assuming the channel consists of 8 bits (which is standard in most formats)
    binary[-1] = msg_bit

    new_pixel = red_channel, np.uint8(int(''.join(binary), 2)), green_channel 

    return new_pixel
    

Now we'll call this function on all pixels, until the message is encoded.

In [186]:
def encode_message(pixels, msg):
    # side effects: 'pixels' will be altered direcly

    bit_index = 0

    for row in pixels:
        for pixel in row:
            bit = msg[bit_index]
            # print(f"before: {pixel}")
            pixel = encode_on_pixel(pixel, bit)
            # print(f"after: {pixel}")
            bit_index += 1
            if bit_index >= len(msg): break
        if bit_index >= len(msg): break

    return pixels

After encoding the message, we just need to **store** it:

In [187]:
def store_image(pixels, file_name : str):
    # side effects: saves image

    image = Image.fromarray(pixels, 'RGB')
    image.save(file_name)

But of course, we also need to be able to **decode the message**. Otherwise, all this work was for nothing. 

In [ ]:
def decode_pixel(pixel):
    red_channel, blue_channel, green_channel = pixel
    binary = list(f'{blue_channel:08b}') # Assuming the channel consists of 8 bits (which is standard in most formats)
    return binary[-1]


def decode_message(pixels):
    msg = []
    msg_length = 30*8 # implement a way to communicate this instead

    for row in pixels:
        for pixel in row:
            msg.append(decode_pixel(pixel))
            if len(msg) >= msg_length: break
        if len(msg) >= msg_length: break
    
    return msg

msg = "War and hate, war and hate!"

encoded = str_to_bin(msg)
pixels = load_image("nomorefanmail.jpg")
new_pixels = encode_message(pixels, encoded)
store_image(new_pixels, "secret.jpg")

bin_decoded = decode_message(new_pixels)
decoded = bin_to_str(bin_decoded)

src_bin_decoded = decode_message(pixels)
src_decoded = bin_to_str(src_bin_decoded)

print(f"secret message:  {msg}\ndecoded in text: {decoded} \ndecoded src image: {src_decoded}")
print(f"encoded in binary:  {''.join(encoded)}\ndecoded from image: {''.join(bin_decoded)} \ndecoded src image:  {''.join(src_bin_decoded)}")

secret message:  War and hate, war and hate!
decoded in text: * gÃà4îçø0îyI-$³åËÌKà%Ã¤ 
decoded src image: * gÃà4îçø0îyI-$³åËÌKà%Ã¤
encoded in binary:  010101110110000101110010001000000110000101101110011001000010000001101000011000010111010001100101001011000010000001110111011000010111001000100000011000010110111001100100001000000110100001100001011101000110010100100001
decoded from image: 001010100001100000000000011001111100001111100000100110010011010011101110111001111111100000011111001100001110111001111001010010010010110100010010001001001011001111100101000110001100101111001100010010111110000000100101110000110001001010100100 
decoded src image: 001010100001100000000000011001111100001111100000100110010011010011101110111001111111100000011111001100001110111001111001010010010010110100010010001001001011001111100101000110001100101111001100010010111110000000100101110000110001001010100100


Finally, we end up with the following: